In [2]:
import numpy as np
from pymongo import MongoClient
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from urllib.request import Request, urlopen

In [3]:
class FoxScraper():
    
    def __init__(self):
        self.name = "Fox News Web Scraper"
        self.page = 'https://www.foxnews.com/politics'
        
    def get_urls(self):
        self.links=[]
        count=0
        pg = requests.get(self.page)
        if pg.status_code==200:
            soup = BeautifulSoup(pg.text,"xml")
            data = soup.findAll('h4', {'class':"title"})
        else:
            print("Failed to get page")

        for i in data:
            link= ( i.find('a').get("href"))
            if link not in self.links and link.startswith('https://video')==False:
                self.links.append('https://www.foxnews.com' + link)
        #for link in links:
            #urls.insert_one({'link': link})
        return self.links
    
    def get_info(self):
        client = MongoClient()
        self.fox_db = client['fox']
        self.urls = self.fox_db['urls']
        self.articles = self.fox_db['articles']
        for link in self.links:
            pg = requests.get(link)
            if pg.status_code == 200:
                soup = BeautifulSoup(pg.text,"xml")
                data = soup.findAll('div', {'class':"article-body"})
                content = []
                for i in data:
                    for j in i.find_all('p'):
                        t = j.get_text()
                        content.append(t)
                content = " ".join(content)
                title = soup.find('h1').text
                key = {"title":title}
                value = {'title':title, 'content': content}
                (self.articles).update(key,value,upsert=True)
                time.sleep(2)
                
  
        

In [60]:
class ReutersScraper():
    
    def __init__(self):
        self.name = "Reuters Web Scraper"
        self.page = 'https://www.reuters.com/politics'
        
    def get_urls(self):
        self.links=[]
        count=0
        pg = requests.get(self.page)
        
        req = Request(self.page, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage,"lxml")
        data = soup.findAll('h2', {'class':"FeedItemHeadline_headline"})
        

        for i in data:
            link= ( i.find('a').get("href"))
            if link not in self.links:
                self.links.append(link)
        return self.links
    
    def get_info(self):
        client = MongoClient()
        self.reuters_db = client['reuters']
        self.urls = self.reuters_db['urls']
        self.articles = self.reuters_db['articles']
        for link in self.links:
            req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            soup = BeautifulSoup(webpage,'lxml')
            data = soup.findAll('div', {'class':"StandardArticleBody_body"})
            content = []
            for i in data:
                for j in i.find_all('p'):
                    t = j.get_text()
                    content.append(t)
            content = " ".join(content)
            title = soup.find('h1', {"class" : "ArticleHeader_headline"}).text
            key = {"title":title}
            value = {'title':title, 'content': content}
            (self.articles).update(key,value,upsert=True)
            time.sleep(2)
            
                

